# MagPySV example workflow - high latitude observatories

# Setup

In [ ]:
# Setup python paths and import some modules
from IPython.display import Image
import sys
sys.path.append('..')
import os
import datetime as dt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
# Import all of the MagPySV modules
import magpysv.denoise as denoise
import magpysv.io as io
import magpysv.model_prediction as model_prediction
import magpysv.plots as plots
import magpysv.tools as tools

In [ ]:
%matplotlib notebook

# Downloading data

In [ ]:
from gmdata_webinterface import consume_webservices as cws

# Required dataset - only the hourly WDC dataset is currently supported 
cadence = 'hour'
service = 'WDC'

# Start and end dates of the data download
start_date = dt.date(1980, 1, 1)
end_date = dt.date(2010, 12, 31)

# Observatories of interest
observatory_list = ['BLC', 'BRW', 'MBC', 'OTT', 'RES', 'STJ', 'THL', 'VIC', 'YKC']

# Output path for data
download_dir = 'data'

cws.fetch_data(start_date= start_date, end_date=end_date,
        station_list=observatory_list, cadence=cadence,
        service=service, saveroot=download_dir)

# Initial processing

Extract all data from the WDC files, convert into the proper hourly means using the tabular base and save the X, Y and Z components to CSV files. This may take a few minutes.

In [ ]:
io.wdc_to_hourly_csv(wdc_path=download_dir, write_dir=os.path.join(download_dir, 'hourly'), obs_list=observatory_list,
                  print_obs=True)

In [ ]:
# Path to file containing baseline discontinuity information
baseline_data = tools.get_baseline_info(fname='baseline_records')

In [ ]:
# Loop over all observatories and calculate SV series as annual differences of monthly means (ADMM) for each
for observatory in observatory_list:
    print(observatory)
    # Load hourly data
    data_file = observatory + '.csv'
    hourly_data = io.read_csv_data(
        fname=os.path.join(download_dir, 'hourly', data_file),
        data_type='mf')

    # Discard days with Ap > threshold (where Ap is the daily average of the 3-hourly ap values) - optional,
    # uncomment the next two lines
#    hourly_data = tools.apply_Ap_threshold(obs_data=hourly_data, Ap_file=os.path.join('index_data', 'ap_daily.csv'),
#                               threshold=30.0)

    # Resample to monthly means
    resampled_field_data = tools.data_resampling(hourly_data, sampling='MS', average_date=True)
    # Correct documented baseline changes
    tools.correct_baseline_change(observatory=observatory,
                          field_data=resampled_field_data,
                          baseline_data=baseline_data, print_data=True)
    # Write out the monthly means for magnetic field
    io.write_csv_data(data=resampled_field_data,
                            write_dir=os.path.join(download_dir, 'monthly_mf'),
                            obs_name=observatory)
    # Calculate SV from monthly field means
    sv_data = tools.calculate_sv(resampled_field_data,
                                   mean_spacing=12)
    # Write out the SV data
    io.write_csv_data(data=sv_data,
                               write_dir=os.path.join(download_dir, 'monthly_sv', 'admm'),
                               obs_name=observatory)

# High latitude regions

In [ ]:
from IPython.display import Image
Image("zonemap.png")

Rerun the analysis below for each of the three high latitude regions. Besides the Setup section, everything preceding this cell only needs be run only once.

## Concatenate the data for our selected observatories

Select observatories in one high latitude region.

In [ ]:
observatory_list = ['MBC', 'RES', 'THL'] # Polar cap
#observatory_list = ['BLC', 'BRW', 'YKC'] # Auroral zone
#observatory_list = ['OTT', 'STJ', 'VIC'] # Sub-auroral zone

Concatenate the data for our selected observatories.

In [ ]:
# Where the data are stored
download_dir = 'data'

# Start and end dates of the analysis as (year, month, day)
start = dt.datetime(1980, 1, 1)
end = dt.datetime(2010, 12, 31)

obs_data, model_sv_data, model_mf_data = io.combine_csv_data(
    start_date=start, end_date=end, obs_list=observatory_list,
    data_path=os.path.join(download_dir, 'monthly_sv', 'admm'),
    model_path='model_predictions', day_of_month=15)

dates = obs_data['date']

In [ ]:
obs_data

# SV plots

In [ ]:
for observatory in observatory_list:
    fig = plots.plot_sv(dates=dates, sv=obs_data.filter(regex=observatory),
                    model=model_sv_data.filter(regex=observatory),
                    fig_size=(6, 6), font_size=10, label_size=16, plot_legend=False,
                    obs=observatory, model_name='COV-OBS')

# Outlier detection

Optionally remove spikes in the data before denoising. Large outliers can affect the denoising process so better to remove them beforehand for some series (i.e. at high latitude observatories). Try changing the threshold or window length to see how this affects which points are identified as outliers.  

In [ ]:
obs_data.drop(['date'], axis=1, inplace=True)
for column in obs_data:    
    obs_data[column] = denoise.detect_outliers(dates=dates, signal=obs_data[column], obs_name=column,
                                               threshold=4,
                                               window_length=120, plot_fig=True, fig_size=(10,3))
obs_data.insert(0, 'date', dates)

# Residuals

To calculate SV residuals, we need SV predictions from a geomagnetic field model. This example uses output from the COV-OBS model by Gillet et al. (2013, Geochem. Geophys. Geosyst.,
https://doi.org/10.1002/ggge.20041; 2015, Earth, Planets and Space,
https://doi.org/10.1186/s40623-015-0225-z2013) to obtain model
predictions for these observatory locations. The code can be obtained from
http://www.spacecenter.dk/files/magnetic-models/COV-OBSx1/ and no modifications
are necessary to run it using functions found MagPySV's model_prediction module. For convenience, model output for the locations used in this notebook are included in the examples directory.

In [ ]:
residuals = tools.calculate_residuals(obs_data=obs_data, model_data=model_sv_data)

In [ ]:
model_sv_data.drop(['date'], axis=1, inplace=True)
obs_data.drop(['date'], axis=1, inplace=True)

# External noise removal

Compute covariance matrix of the residuals (for all observatories combined) and its eigenvalues and eigenvectors. Since the residuals represent signals present in the data, but not the internal field model, we use them to find a proxy for external magnetic fields (Wardinski & Holme, 2011, GJI, https://doi.org/10.1111/j.1365-246X.2011.04988.x).

In [ ]:
denoised, proxy, eigenvals, eigenvecs, projected_residuals, corrected_residuals = denoise.eigenvalue_analysis(
    dates=dates, obs_data=obs_data, model_data=model_sv_data, residuals=residuals,
    proxy_number=1)

# Denoised SV plots

Plots showing the original SV data, the denoised data (optionally with a running average) and the field model predictions.

In [ ]:
for observatory in observatory_list:
    xratio, yratio, zratio = plots.plot_sv_comparison(dates=dates, denoised_sv=denoised.filter(regex=observatory),
        residuals=residuals.filter(regex=observatory),
        corrected_residuals = corrected_residuals.filter(regex=observatory),
        noisy_sv=obs_data.filter(regex=observatory), model=model_sv_data.filter(regex=observatory),
        model_name='COV-OBS',
        fig_size=(6,6), font_size=10, label_size=14, obs=observatory, plot_rms=True)

Plots showing the denoised data (optionally with a running average) and the field model predictions.

In [ ]:
for observatory in observatory_list:
    plots.plot_sv(dates=dates, sv=denoised.filter(regex=observatory), model=model_sv_data.filter(regex=observatory),
                  fig_size=(6, 6), font_size=10, label_size=14, plot_legend=False, obs=observatory,
                  model_name='COV-OBS')

# Plot proxy signal, eigenvalues and eigenvectors

Compare the proxy signal used to denoise the data with a geomagnetic index at the same temporal resolution. Dst measures the intensity of the equatorial electrojet (the "ring current"). AE measures the intensity of the auroral electrojet. Files included with this notebook for annual differences: dst_admm.csv, ap_admm_dst and ae_admm.csv

In [ ]:
plots.plot_index_dft(index_file=os.path.join('index_data', 'dst_admm.csv'), dates=denoised.date, signal=proxy.astype('float'),
                     fig_size=(6, 6), font_size=10, label_size=14, plot_legend=True, index_name='Dst')

Plot the eigenvalues of the covariance matrix of the residuals

In [ ]:
plots.plot_eigenvalues(values=eigenvals, font_size=12, label_size=16, fig_size=(6, 3))

Plot the eigenvectors corresponding to the three largest eigenvalues. The noisiest direction (v_000, used to denoise in this example) is mostly:
Z in the polar region (no strong correlation with Dst, AE or Dst indices), X and Z in auroral zone (correlates with the AE index) and X and Z in the sub-auroral zone (correlates with the Dst index, similar to European observatories)

In [ ]:
plots.plot_eigenvectors(obs_names=observatory_list, eigenvecs=eigenvecs[:,0:3], fig_size=(6, 4),
                          font_size=10, label_size=14)

# Outlier detection

Remove remaining spikes in the time series (if needed).

In [ ]:
denoised.drop(['date'], axis=1, inplace=True)
for column in denoised:
    denoised[column] = denoise.detect_outliers(dates=dates, signal=denoised[column], obs_name=column, threshold=5,
                                               window_length=120, plot_fig=False, fig_size=(10, 3), font_size=10,
                                               label_size=14)
denoised.insert(0, 'date', dates)

# Write denoised data to file

In [ ]:
for observatory in observatory_list:
    print(observatory)
    sv_data=denoised.filter(regex=observatory)
    sv_data.insert(0, 'date', dates)
    sv_data.columns = ["date", "dX", "dY", "dZ"]
    io.write_csv_data(data=sv_data, write_dir=os.path.join(download_dir, 'denoised', 'highlat'),
                               obs_name=observatory, decimal_dates=False)